***1. Quel est le principe général des algorithmes de hachage ?***

Hachage doit fonctionne unidirectionelle, 

***2. Quel est la longueur en bits d'un haché généré par MD5 ? Même question si le haché est exprimé en hexadécimal ?***

Le hachange MD5  genere un haché de 128 bits et 32 hexadecimals

***3. Pourquoi MD5 ne doit plus être utilisé pour stocker les mots de passe ? Que faut-il utiliser maintenant ? Dans quel cas MD5 pourrait éventuellement être encore utilisé ?***

Le hachage de MD5 est vulnerable d'attaque de collisions. Pour stocker le mot de passe, on utilise l'algorithme de hachage SHA256. MD5 est toujours utilisé pour vérifier intégralité de fichier.

***4. Quelles sont les trois méthodes qui existent pour attaquer les empreintes générées par un
algorithme de hachage ?***

Les trois attaques sont attaque par dictionnaire, force brute et rainbow tables.

***Exercice 2 : Algorithme MD5***


2. Tester la fonction hashMD5 avec la chaîne : « bergere » (sans accent). Vous devez obtenir le
haché : « 6484846ceffdc51484af2b549ffa21fd ».
3. Modifier juste un caractère de la chaîne en mettant l'accent grave à la place du second 'e' : le
haché doit être complètement différent.

In [8]:
import hashlib
def hashMD5(text):
    byte = text.encode("utf-8")
    result = hashlib.md5(byte).hexdigest()
    return result


print(hashMD5("bergere"))
print(hashMD5("bérgere"))

6484846ceffdc51484af2b549ffa21fd
0a50898c15b84d0bd8b29ad4006cc859


4. On souhaite trouver les collisions d’un haché généré avec MD5.
Écrire la fonction collisionsMD5 (ch, nomFic) où ch est une chaîne représentant une
empreinte (ou une portion d’empreinte) et nomFic est le fichier contenant les mots à tester.
Cette fonction devra pour chaque mot contenu dans nomFic générer son empreinte MD5 et la
comparer à ch. collisionsMD5 retournera donc la liste des mots dont l’empreinte MD5 est
égale à ch.
Attention : Pour simplifier l’exercice, la chaîne ch passée en paramètre pourra être plus
courte que les empreintes générées par MD5. Les empreintes à retourner sont donc celles
qui commencent par ch. Elles ne sont pas forcément entièrement égales.

In [28]:
def collisionsMD5 (ch, nomFic) :
    listres = []
    with open(nomFic,"r") as file:
        for i in file:
            i = i.strip()
            result = hashMD5(i)
            if result.startswith(ch):
                listres.append(i)
    return listres


print(collisionsMD5("cc24c","wordlist.txt"))

['willingly', 'zonary']


***Exercice 3 : Cracker les mots de passe de Monsieur Petit – Étape 1***
    
Vous avez réussi à récupérer une table utilisée par les programmes de Monsieur Petit. Elle
contient les informations d’authentification des utilisateurs :

***1. Quel algorithme semble avoir été utilisé ? Justifier votre réponse.***

C'est un hachage de type SHA256 parce que le resultat est 64 hexadecimal

***2. Quelle est l’utilité du champ Salt ? Comment ces valeurs ont pu être utilisées pour générer
une empreinte ?***

Salt est utilisé pour ajouter le mdp avant de hacher.

In [81]:
def hashsha256(text):
    byte = text.encode("utf-8")
    result = hashlib.sha256(byte).hexdigest()
    return result

def decryptPwdExo3(valhash,salt,nomFic):
    with open (nomFic,"r") as file:
        for i in file:
            i = i.strip()
            test = i + salt
            result = hashsha256(test)
            if result.startswith(valhash):
                return i
    
def hashsha384(text):
    byte = text.encode("utf-8")
    result = hashlib.sha384(byte).hexdigest()
    return result

hashHarbiang = "32c4de3eeb8b04fc3326b4595c3cff93e81ac2d1e92f37b4241ad2d3ef40ff67"
print(decryptPwdExo3(hashHarbiang, "471","wordlist.txt"))

hashDile = "faa549aa1c54393af8ded83e192b7cda5d414f9f73668fba1580ff68223854ad"
print(decryptPwdExo3(hashDile, "382","wordlist.txt"))


print(hashsha256("H"))
print(hashsha256("Hregency"))
print(hashsha256("49"))
print(hashsha256("Hishand"))
print(hashsha256(hashMD5("regency")))

print(decryptPwdExo3("6454197dccd315db04e44122cb6dad26964cec0c487cef5f71492a0e9ce1922877718e", "Daylloween1983","wordlist.txt"))

regency
falsely
44bd7ae60f478fae1061e11a7739f4b94d1daf917982d33b6fc8a01a63f89c21
5c729a466d14f72509ee813d07d9186584e6e5d24beefc0c378a3e59ae0049be
0e17daca5f3e175f448bacace3bc0da47d0655a74c8dd0dc497a3afbdad95f1f
b2501d6ceacb528aa63bdf045c7c053ab8b5bd4e957053a6238c3e1f82b1df1e
e67fd0dfb84b1cc94f5ade75345e93963ddd04203285fa13a09764639f11f8cc
None


***4. Quelle est la faille ? Pourquoi l’attaque a été si simple à mettre en œuvre ?***

Le sel est connu

***Exercice 4 : Cracker les mots de passe de Monsieur Petit – Étape 2***
    
Monsieur Petit a (un peu) sécurisé ses données. La nouvelle version de la table des utilisateurs ne
contient plus que les champs Username et Passwd (Salt a disparu) :

Monsieur Petit a quelque peu modifié sa façon de faire pour générer les empreintes.

***1. Après avoir attentivement regardé les valeurs du champ Passwd, essayer de trouver la
logique de Monsieur Petit.***

***2. Écrire la fonction decryptPwdExo4 qui implémente la réponse à la question précédente. Bien
entendu, cette fonction prendra au moins une empreinte et le nom du fichier contenant la liste
de mots en paramètres. Elle retournera le mot à l’origine de l’empreinte.
Tester decryptPwdExo4 pour trouver le mot de passe de l’utilisateur Daylloween1983 (tarot)
et celui de Hommat95 (grand).***

In [101]:
def decryptPwdExo4 (valhash, nomFic):
    first = valhash[:6]
    
    with open(nomFic,"r") as file:
         for i in file:
            i = i.strip()
            test = i + first
            result = hashsha256(test)
            if valhash == first + result:
                return i
            
hashDay = "33487971dc94f21be79158e1d9a8d62312a726eaa047f3b678d862f3d397f570c3aceb"
print(decryptPwdExo4 (hashDay, "wordlist.txt"))

hashHammot = "9678753e0b37c064cd7d17a16c6c8eacac96ae9bc7c02fa099069698106716d1dcb4bd"
print(decryptPwdExo4 (hashHammot, "wordlist.txt"))

tarot
grand


***3. Pourquoi est-ce toujours aussi simple de retrouver les mots de passe à partir des informations
stockées dans la table ?***

***Exercice 5 : Cracker les mots de passe de Monsieur Petit – Étape 3***
    
Monsieur Petit pense avoir trouvé un moyen infaillible pour stocker les mots de passe. Il a utilisé
des informations qui lui sont personnelles pour chiffrer les mots de passe. Voici la dernière
version de la table :

***1. L’idée de Monsieur Petit n’est pas si mauvaise ... mais il n’aurait pas dû en dire trop sur sa
page Web ! Après avoir compris la dernière idée de Monsieur Petit, écrire la fonction
decryptPwdExo5 qui vous permettra de déchiffrer les mots de passe.
La tester en faisant afficher les mots de passe de l’utilisateur Whistre (trottoir) et celui de
Acep1956 (imitation).***

In [122]:
def decryptPwdExo5(valhash,salt,nomFic):
    with open(nomFic,"r") as file:
         for i in file:
            i = i.strip()
            test = salt + i
            result = hashsha256(test)
            if valhash == result:
                return i

    
print(decryptPwdExo5("46c74e72bb787c4e24ad7c4c039d383d725c453e4a0e0fbef844283ead1b1da4", "02468", "wordlist.txt"))

imitation
